In [65]:
import os

In [66]:
import sys
sys.path.append('c:\\Users\\Sadeeq\\Desktop\\NLP Projects\\Text-Summarizer-Project\\src')

In [67]:
import sys
project_path = 'c:\\Users\\Sadeeq\\Desktop\\NLP Projects\\Text-Summarizer-Project'
if project_path not in sys.path:
    sys.path.append(project_path)

In [68]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list

In [69]:
from src.textSummarizer.utils.common import read_yaml, create_directories
from src.textSummarizer.constant import *

In [70]:
from textSummarizer.constant import CONFIG_FILE_PATH, PARAMS_FILE_PATH
new_config_path = Path('../') / CONFIG_FILE_PATH
new_params_path = Path('../') / PARAMS_FILE_PATH

In [89]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = new_config_path,
        params_filepath = new_params_path):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            ALL_REQUIRED_FILES = config.ALL_REQUIRED_FILES,
        )
        return data_validation_config


In [90]:
from textSummarizer.logging import logger

In [91]:
import os

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = None
            
            # Path to the 'data_ingestion' directory inside the 'artifacts' directory
            ingestion_path = os.path.join("..", "artifacts", "data_ingestion", "samsum_dataset")
            if not os.path.exists(ingestion_path):
                raise FileNotFoundError(f"The path {ingestion_path} does not exist.")
            
            # Path to the 'data_validation' directory inside the 'artifacts' directory
            validation_path = os.path.join("..", "artifacts", "data_validation")
            if not os.path.exists(validation_path):
                os.makedirs(validation_path)  # This will create the 'data_validation' directory if it doesn't exist

            all_files = os.listdir(ingestion_path)
            
            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                else:
                    validation_status = True
                
                # Ensure STATUS_FILE is just a filename, like 'status.txt'
                status_file_path = os.path.join(validation_path, os.path.basename(self.config.STATUS_FILE))
                with open(status_file_path, 'w') as f:
                    f.write(f"validation status: {validation_status}")
            
            return validation_status
        
        except Exception as e:
            raise e


In [92]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()  
except Exception as e:
    raise e

[2024-03-03 01:42:44,735: INFO: common: yaml file: ..\config\config.yaml loaded successfully]
[2024-03-03 01:42:44,738: INFO: common: yaml file: ..\params.yaml loaded successfully]
[2024-03-03 01:42:44,738: INFO: common: Directory artifacts created successfully]
[2024-03-03 01:42:44,739: INFO: common: Directory artifacts/data_validation created successfully]
